In [194]:
import os
import numpy as np
import random
import math
from collections import Counter

In [195]:
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    dictionary_temp = Counter(all_words)

    # Paste code for non-word removal here(code snippet is given below)
    list_to_remove = dictionary_temp.keys()
    for item in list_to_remove:
        if item.isalpha() == False:    #Determine whether it is punctuation
            del dictionary[item]
        elif len(item) == 1:           #
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary

In [196]:
def extract_features(root_dir,dictionary):
    emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    all_words = []
    features_matrix = np.zeros((len(emails),3000))
    docID = 0
    for mail in emails:
        print(mail)
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1
    return features_matrix
    

In [197]:
Path = "train-mails"
dir = make_Dictionary(Path)
train_matrix = extract_features(Path,dir)
print(train_matrix)

Path_test = "test-mails"
dir_test = make_Dictionary(Path_test)
test_matrix = extract_features(Path_test,dir_test)
print(test_matrix)

3msg1.txt
train-mails\6-31msg1.txt
train-mails\6-338msg1.txt
train-mails\6-33msg1.txt
train-mails\6-36msg1.txt
train-mails\6-380msg1.txt
train-mails\6-381msg1.txt
train-mails\6-381msg2.txt
train-mails\6-381msg3.txt
train-mails\6-381msg4.txt
train-mails\6-381msg5.txt
train-mails\6-382msg1.txt
train-mails\6-392msg1.txt
train-mails\6-41msg1.txt
train-mails\6-421msg1.txt
train-mails\6-425msg2.txt
train-mails\6-425msg3.txt
train-mails\6-42msg1.txt
train-mails\6-42msg2.txt
train-mails\6-42msg3.txt
train-mails\6-430msg1.txt
train-mails\6-430msg2.txt
train-mails\6-430msg3.txt
train-mails\6-430msg4.txt
train-mails\6-430msg5.txt
train-mails\6-453msg0.txt
train-mails\6-453msg1.txt
train-mails\6-453msg2.txt
train-mails\6-454msg1.txt
train-mails\6-455msg1.txt
train-mails\6-455msg2.txt
train-mails\6-45msg1.txt
train-mails\6-473msg1.txt
train-mails\6-474msg1.txt
train-mails\6-4msg1.txt
train-mails\6-4msg2.txt
train-mails\6-4msg3.txt
train-mails\6-50msg0.txt
train-mails\6-50msg1.txt
train-mails\6-50ms

In [206]:
class NaiveBayes:
    Mean = {}
    Std = {}
    def fit(self,x,y):
        #divide x by x
        DivideX = {}
        for i in range(len(y)):
            if (y[i] not in DivideX):
                DivideX[y[i]]=[]
                DivideX[y[i]].append(x[i])
            else:
                DivideX[y[i]] = np.vstack((DivideX[y[i]],x[i]))
        for key in DivideX:
            self.Mean[key] = []
            self.Std[key] = []
            for i in range(DivideX[key].shape[1]):
                self.Mean[key].append(np.mean(DivideX[key][:,i]))
                if np.var(DivideX[key][:,i]) == 0:
                    self.Std[key].append(0.00001)
                else:
                    self.Std[key].append(np.std(DivideX[key][:,i]))
        # print(self.Mean)
    def probabilty(self,data,key):
        prob = 1
        # print(data)
        for i in range(len(data)):
            if self.Std[key][i] == 0:
                continue
            exp = math.exp(-(math.pow(data[i] - self.Mean[key][i],2)/(2*math.pow(self.Std[key][i],2))))
            prob *= (1/(math.sqrt(2*math.pi)*self.Std[key][i])) * exp
        return prob
    def predict(self,data):
        result = []
        for i in range(data.shape[0]):
            MaxProb = 0
            MaxProb_key = 0
            for key in self.Mean:
                Prob_temp = self.probabilty(data[i],key)
                if(Prob_temp > MaxProb):
                    MaxProb = Prob_temp
                    MaxProb_key = key
            result.append(MaxProb_key)
        return result
    def accuracy(self,data,prediction):
        correct = 0
        for x in range(len(data)):
            if data[x] == prediction[x]:
                correct += 1
        return (correct/float(len(data))) * 100


In [207]:
NB = NaiveBayes()
train_labels = np.zeros(train_matrix.shape[0])
train_labels[351:701] = 1
NB.fit(train_matrix,train_labels)
pred = NB.predict(train_matrix)
print(NB.accuracy(train_labels,pred))

89.88603988603988


In [208]:
pred_test = NB.predict(test_matrix)
test_label = np.zeros(test_matrix.shape[0])
test_label[130:] = 1
print(pred_test)
print(NB.accuracy(test_label,pred_test))

[0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 1.0, 0, 0, 0.0, 0, 0, 1.0, 1.0, 1.0, 0, 0, 1.0, 1.0, 0.0, 0.0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0.0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
52.69230769230769


In [209]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
GNB = GaussianNB()
print(test_label)
GNB.fit(train_matrix,train_labels)
y_pred = GNB.predict(train_matrix)
print(NB.accuracy(train_labels,y_pred))
y_pred_test = GNB.predict(test_matrix)
print(NB.accuracy(test_label,y_pred_test))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
98.14814814814815
54.230769230769226
